In [ ]:
#r "nuget: xunit, 2.8.1"
using System.Threading;
using Xunit;

class Message
{
    static int messageHandled = 0;
    int id;
    public Message()
    {
        id = Thread.CurrentThread.ManagedThreadId;
    }

    public void Handle()
    {
        Interlocked.Increment(ref messageHandled);
        Assert.NotEqual(id, Thread.CurrentThread.ManagedThreadId);
    }

    public static void Check()
    {
        Assert.Equal(200, messageHandled);
    }
}

In [ ]:
using System.Collections.Concurrent;
using System.Threading;
using System.Threading.Tasks;

var queue1 = new BlockingCollection<Message>(2); //очередь первого потока
var queue2 = new BlockingCollection<Message>(2); //очередь второго потока


// Создаем два потока
var thread1 = new Thread(() =>
{
    for (int i = 0; i < 100; i++)
    {
        // Читаем сообщение из очереди первого потока
        var message = queue1.Take();
        // Обрабатываем сообщение
        message.Handle();
        // Создаем новое сообщение и записываем его в очередь второго потока
        var newMessage = new Message();
        queue2.Add(newMessage);
    }
    
});
var thread2 = new Thread(() =>
{
    for (int i = 0; i < 100; i++)
    {
        // Читаем сообщение из очереди второго потока
        var message = queue2.Take();
        // Обрабатываем сообщение
        message.Handle();
        // Создаем новое сообщение и записываем его в очередь первого потока
        if(i!=99)
        {
            var newMessage = new Message();
            queue1.Add(newMessage);
        }
        
    }
    
});
// Создаем первое сообщение и записываем его в очередь первого потока
var firstMessage = new Message();
queue1.Add(firstMessage);

// Запускаем потоки
thread1.Start();
thread2.Start();

// Дожидаемся завершения работы потоков
Parallel.Invoke(()=>
{
    thread1.Join();
},
()=>
{
    thread2.Join();
}
);



In [ ]:
Assert.Equal(0, queue1.Count);
Assert.Equal(0, queue2.Count);
Message.Check();